# Topic 2 -- Logistic Regression

Welcome to topic 2 of the Beginner AI Course! In this notebook, we are going to explore the **classification** problem in supervised learning, how **logistic regression** triumphs over **linear regression** in this area, as well as see logistic regression in action predicting the types of Pokemon!

## Table of Contents
1. [Installing Dependencies](#installing)


2. [Linear Regression vs Classification](#linregfail)


3. [Logistic Regression](#logreg)
    - [Sigmoid Activation Function](#sigmoid)
    - [Cost Function](#cost)
    - [Putting it all together](#putting)
   
   
4. [Binary Classification](#binary)
    - [Pokemon Classification Problem](#pokemon)
    - [Choosing our Features](#choosing)
    - [Imbalanced Classes](#imbalanced)
    - [Training our Logistic Regression Model](#training1)
    - [Confusion Matrix](#conf)
    - [Observations](#obs1)
    
    
5. [Multi-Class Classification](#multi)
    - [One vs All](#onevall)
    - [One-Hot Encoding](#onehot)
    - [Choosing our Features](#choosingf)
    - [Choosing our Labels](#choosingl)
    - [Training a Multi-Class Classifier](#train_m)
    - [Observations](#obs2)

## Installing Dependencies <a name="installing">

Lets first install the dependencies needed for this demo. Here are a list of the dependencies and descriptions on what they are for:

- **Numpy**: Powerful linear regression library
- **Pandas**: Used for data visualization and organization
- **SciKitLearn**: Maching learning library containing the `LogisticRegression` class
- **Bokeh**, **MatPlotLib**, and  **SeaBorn**: Plots and Graphs for visualizing the data
- **disp_utils**: A custom module for displaying the demo visuals

In [ ]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
import matplotlib.pyplot as plt
import seaborn as sn
from disp_utils import *

pd.set_option('display.max_columns', None)

## Linear Regression vs Classification <a name="linregfail">

Lets set the scene: you are trying to train a machine learning model that will predict whether you get admitted into the **NBA** based on your **height**. Notice that this is a classification problem, as there are two discrete values that the model is allowed to output: Either **yes**, you get admitted, or **no**, you are rejected. We will denote $\hat{y}$ as the output, and it could only take on **1 or 0**, 1 being yes and 0 being no. Lets display a plot of what a hypothetical dataset would look like:

In [ ]:
display_height_dataset()

Here, we can see that at around 1.8m, you will be admitted into the NBA. After running linear regression, we are able to fit a linear hypothesis function to the data:

In [ ]:
display_height_line1()

One thing to note is that the linear hypothesis function is not discrete, which is okay, because we can assign the output value of the function as a **probability** that you will be admitted given your height. For example, if you are 1.6 metres tall, there is roughly a 10% probability that you will be admitted, whereas if you are 2 meters tall, there is a 90% probability you will be admitted. We will use this probability metric to make a cut-off: **all probabilities greater or equal to 50% we will predict $\hat{y} = 1$, and all probabilities less than 50% we will predict $\hat{y} = 0$.** 

However, there are a few problems to discuss: if your height is less than 1.55 metres, then your probability of getting admitted is less than 0%, and if your height is over 2.05 metres, your probability is greater than 100%. Furthermore, lets say we add some **outliers**; lets add the height of 1.0 metres to our dataset:

In [ ]:
display_height_line2()

Here, we can see that adding an outlier has significantly shifted the **decision boundary**. Before, a height of 1.8 meters and above will have a high likelyhood of getting admitted, whereas now, with the introduction of one datapoint, the acceptable height has decreased to 1.67m.

**Therefore, Linear Regression cannot be used for classification problems.**

## Logistic Regression <a name="logreg">

Despite the name "regression", logistic regression is an algorithm used for classification problems. The base concept is **very similar to linear regression**; you still have a linear feed forward step. This time I will use the letter $z$ to denote the linear output:

<div style="text-align: center">
    <div>
    &nbsp;
    </div>
 $z = wx + b$
</div>

### Sigmoid Activation Function <a name="sigmoid">
Now, however, we input the linear results into the **sigmoid function**. The sigmoid function is one type of **activation function** that bounds the output between 0 and 1, which eliminates probabilities of greater than 1 or less than 0. Also, using the sigmoid activation function ensures that the **decision boundary** would not get shifted due to **outliers**. 
    
The sigmoid function is represented by the formula down below:

<div style="text-align: center">
    <div>
    &nbsp;
    </div>
 $\begin{align}\sigma(z) = {1\over 1 + e^{-z}}\end{align}$
</div>
    
To visualize the sigmoid function, lets create a sigmoid function down below:

In [ ]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [ ]:
display_func(sigmoid)

### Cost Function <a name="cost">
    
Previously, we learned that Linear Regression uses the **Mean Squared Error (MSE)** cost function to measure the error in the fit. While MSE works great for linear regression, it does not work very well for logistic regression. This is because due to the **sigmoid function** used in logistic regression, the resulting cost function is **non-convex**. Take a look at the figures below showing the non-convex nature of MSE used with the sigmoid function:

In [ ]:
disp_convex()

Non-convex functions can be problematic to **gradient descent**. To make the cost convex, we need a different cost function:

<div style="text-align: center">
    <div>
    &nbsp;
    </div>
 $\begin{align}C(w, b) = {1\over m} \sum\limits_{i=1}^m -y^{(i)}\log(a^{(i)})-(1-y^{(i)})\log(1-a^{(i)})\end{align}$
</div>

This cost function is called **Binary Cross Entropy (BCE)**, and is the cost function of choice for logistic regression. This cost function is guaranteed to be convex when used with the **sigmoid activation function**.

### Putting it all together <a name="putting">

All in all, how logistic regression learns is very similar to linear regression, other than the activation function and the cost function. Here's a summary of how it all works, shown in **pseudo-code**:

<div style="text-align: center">
    <div>
    &nbsp;
    </div>
</div>


```python
# Linear Regression:
for i in range(iterations):
    prediction = w*x + b
    
    cost = MSE_cost_function(prediction, label)
    slopes = cost.get_slopes()
    update(w, b).with(slopes)
    
# Logistic Regression:
for i in range(iterations):
    z = w*x + b
    prediction = sigmoid(z)
    
    cost = BCE_cost_function(prediction, label)
    slopes = cost.get_slopes()
    update(w, b).with(slopes)
```
    

[https://www.desmos.com/calculator/z4petl2oxa](https://www.desmos.com/calculator/z4petl2oxa)
    
[https://www.geogebra.org/3d/yku5x5tf](https://www.geogebra.org/3d/yku5x5tf)
    
---

## Binary Classification <a name="binary">
    
Binary Classification, as the name suggests, predicts on an output space of **two** states. An example of which is NBA Admittance, where the two possible states are either **yes** or **no**. This is the most basic form of logistic regression, and is the basis of more advanced forms of logistic regression.
    
### Pokemon Classification Problem <a name="pokemon">
Lets say we were to build a classifier to classify whether or not a Pokemon is **legendary** or not. This is a perfect example of binary classification. Let's load in the `pokemon.csv` file, which we will use for the rest of this notebook.

In [ ]:
dataset = pd.read_csv('datasets/pokemon.csv')
dataset

Taking a look at the dataset, notice that the rows are **truncated**. We can see the first few examples, as well as the last. Looking at the `is_legendary` column, we notice that all of the legendary pokemons are arranged at the bottom of our dataset. In order get a **uniform distribution** of either class in the training and testing sets, we need to **shuffle** our dataframe.

In [ ]:
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset.head(10)

### Choosing our Features <a name="choosing">
    
Taking a look at the dataset, there are quite a few features that we can include into our training and testing sets. However, we probably do not need to include all of the features, as many features are redundant. Since we are training a classifier to classify whether or not a Pokemon is **legendary**, we can possibly choose the features `sp_attack`, `sp_defense`, and `weight_kg`.

In [ ]:
features = dataset[['sp_attack', 'sp_defense', 'weight_kg']].to_numpy()
labels = dataset['is_legendary'].to_numpy()

print(f"features shape: {features.shape}")
print(f"labels shape: {labels.shape}")

### Imbalanced Classes <a name="imbalanced">

#### *Intuition*

> *Suppose you were training a classifier for predicting whether a person owns a mansion or not. You gather together a dataset of 100 examples, containing features such as income, occupation, etc. In your dataset, 99 people do not own mansions, and 1 person owns a mansion. You train a classifier, and is impressed by the __98.5%__ accuracy that you obtained! However, I come in and present a model that performs better than yours! I achieved 99% accuracy by __predicting all zeros__. It might get that one instance wrong, but the other 99 instances it is correct.*

Sometimes in the real world, it is **not feasible** to get data with **perfectly balanced classes**. When the classes are not balanced, you will often train models that will be **biased** towards the more frequent class and simply not predict on the less frequent class. Here, we print the number of legendary Pokemons and the number of non-legendary Pokemons.

In [ ]:
# calculates the number of legendary pokemon
num_legendaries = np.sum(dataset['is_legendary'].to_numpy())
# total pokemon
total_pokemon = dataset.shape[0]

print(f"number of legendary pokemon: {num_legendaries}")
print(f"number of non-legendary pokemon: {total_pokemon - num_legendaries}")

The classes in this case are quite **skewed** towards the non-legendary Pokemons in this case. We will resolve that when setting up the logistic regression model.

### Training our Logistic Regression Model <a name="training1">

Training the Logistic Regression model is quite similar to training a Linear Regression model as we have seen before. This time, we are going to use the `LogisticRegressionCV`class instead of the `LinearRegression` class.


In [ ]:
from sklearn.model_selection import train_test_split

# converts all NaN to 0
features = np.nan_to_num(features, nan=0)

X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.1)

Remember the unbalanced classes mentioned above? Here, we are going to pass in the parameter `class_weight='balanced'` into the `LogisticRegressionCV` class, which would automatically balance out the classes by **emphasizing the importance of the less frequent class**.

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import f1_score

logreg = LogisticRegressionCV(max_iter=3000, class_weight='balanced', verbose=5, n_jobs=-1)
poly = PolynomialFeatures(degree=3)


X_train_p = poly.fit_transform(X_train)
X_test_p = poly.fit_transform(X_test)
Y_train_p = Y_train
Y_test_p = Y_test

print(f"X_train_p shape: {X_train_p.shape}")
print(f"Y_train_p shape: {Y_train_p.shape}")
print(f"X_test_p shape: {X_test_p.shape}")
print(f"Y_test_p shape: {Y_test_p.shape}")


In [ ]:
logreg.fit(X_train_p, Y_train_p)

print(f"Accuracy: {logreg.score(X_test_p, Y_test_p)}")

Y_pred_p = logreg.predict(X_test_p)

print(f"F1 Score: {f1_score(Y_test_p, Y_pred_p)}")

### Confusion Matrix <a name="conf">

Often times in classification scenarios, we will evaluate our models by running on the test set and plotting a **confusion matrix**. The **confusion matrix** shows the **ground truth** values on the **horizontal axis** as well as the **predicted values** on the **vertical axis**. The resulting grid then displays frequency of the outcomes of prediction -- whether the label was `True` and the model predicted `False`, or any combination of label values vs prediction values.

In [ ]:
from sklearn.metrics import confusion_matrix

Y_pred = logreg.predict(X_test_p)
Y_true = Y_test_p

conf_mat = pd.DataFrame(confusion_matrix(Y_true, Y_pred, labels=[0, 1]))
sn.heatmap(conf_mat, annot=True)


### Observations <a name="obs1">

Here we see that we obtained an **Accuracy** of around 90%, and an F1 Score of 60%. Notice how we are using these two metrics instead of $R^2$, which was used in linear regression. The accuracy metric measures the number of correct predictions vs the total predictions, and the F1 Score is a combination of precision and recall. Furthermore, the confusion matrix gives us a sense of what actually happened with all the predictions. Let's fill out the observations table down below:


| Model | Observations | Accuracy | F1 |
|:----- |:------------| :--------| :--- |
|Logistic Regression (Binary) | Fairly high accuracy, many false positives in the confusion matrix | 90% | 60% |
    
---

## Multi-Class Classification <a name="multi">

Most of the classification problems nowadays deal with predicting across multiple classes. For example, *is this a picture of a cat, dog, bird, or horse?* One naive way of performing multi-class classification is to assign integer labels to each class:
    
    - cat: 0
    - dog: 1
    - bird: 2
    - horse: 3

This however becomes a **regression problem** -- rather, a **poorly performing** one at best, and since these integers are comparable, there is a chance that the model can learn a `cat`< `dog` < `bird` < `horse` relation. It is worth saying that sklearn allows you to label classes like this, however behind the scenes, it is converting this label encoding into **One-Hot Encoding**
    
### One vs All <a name="onevall">

Consider training 4 Binary Classifiers: the first one predicts on whether it is a picture of a cat or not, the second one predicts whether it is a picture of a dog or not, and so on. This is called **One vs All** classification -- for each classifier, you are setting one class as the output $\hat{y} = 1$, and the rest of the classes as $\hat{y} = 0$.
    
This affirms the idea mentioned above that *Binary Classification forms the basis of more advanced classification methods [multi-class]*. Each class is effectively governed by its own binary classifier, and the final prediction is decided by which ever classifier outputs the **largest value**.
    
### One-Hot Encoding <a name="onehot">
    
When training a multi-class model, since you are effectively training multiple classifiers, each classifier needs an output label. These labels can be stored into an `np.array` as shown:
    
```python
    
# Individual examples
cat = np.array([[1, 0, 0, 0]])
dog = np.array([[0, 1, 0, 0]])
bird = np.array([[0, 0, 1, 0]])
horse = np.array([[0, 0, 0, 1]])
    
# Labels set
Y_train = np.array([
    [1 ,0, 0, 0],  # cat
    [0, 0, 1, 0],  # bird
    [0, 1, 0, 0]   # dog
    ...
])
```
    
This is called **One-Hot Encoding**, where the true class is labeled with a 1, and everywhere else is labeled with a 0.
    
### Choosing our Features <a name="choosingf">
    
We are going to train a Multi-class Logistic Regression Classifier to predict whether the pokemon is **Fire, Water, Grass, or None of the Above (NA)**. First, we will print out the `dataset` to give ourselves a little refresher.

In [ ]:
dataset.head(10)

One thing to ask ourselves when choosing features is *What features are possibly related to the problem?* After all, using `weight_kg` as a feature to predict the type of pokemon is probably not the best idea. Here, we will choose the features that tell us how well a Pokemon of a certain type does **against another type**. For example, an **Ice** type Pokemon might do very well against **Fire**, but not so well against other **Ice** types.

In [ ]:
input_features = list(dataset.columns[1:19])
print(input_features, '\n')
features_m = dataset[input_features].to_numpy()

print(f"features_m shape: {features_m.shape}")

### Choosing our Labels <a name="choosingl">
    
For this classification problem, we want to make **One-Hot encodings** of the types `"Fire", "Water", "Grass", "NA"`. Looking at the `dataset`, we can see the `type1` column containing the primary types of the Pokemon. These are currently stored as datatype `string`, however we want to convert them to integers. `pd.get_dummies()` is a function that will create One-Hot encoding of all the different values of strings it sees in a column and store it as a spearate dataset.

In [ ]:
type1_dummies = pd.get_dummies(dataset[['type1']])
type1_dummies 

From the 18 columns in `type1_dummies`, we will only need 3 columns. These three columns (`"type1_fire, type1_water, type1_grass"`) are stored back into `dataset` as new columns. The `"NA"` column is created via the logic of `not("Fire" or "Water" or "Grass")`.

In [ ]:
dataset['fire'] = type1_dummies['type1_fire']
dataset['water'] = type1_dummies['type1_water']
dataset['grass'] = type1_dummies['type1_grass']
dataset['NA'] = ~(type1_dummies['type1_fire'] | type1_dummies['type1_water'] | type1_dummies['type1_grass']) - 254
dataset.head(10)

In [ ]:
labels_m = dataset[['fire', 'water', 'grass', 'NA']].to_numpy()
print(f"labels_m shape: {labels_m.shape}")

In [ ]:
X_train_m, X_test_m, Y_train_m, Y_test_m = train_test_split(features_m, labels_m, test_size=0.1)

### Training the Multi-Class Classifier <a name="train_m">

Remember how training a multi-class classifier is like training a bunch of binary classifiers based on how many classes you have? When it comes to implementing a multi-class classifier in SKLearn, we will fit a `LogisticRegressionCV()` object **within** a `OneVsRestClassifier()` object, which effectively **creates multiple** `LogisticRegressionCV()` classifiers for binary classification of each class.

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

logreg_m = OneVsRestClassifier(LogisticRegressionCV(max_iter=2000, class_weight='balanced'), n_jobs=-1)

logreg_m.fit(X_train_m, Y_train_m)
accuracy = logreg_m.score(X_test_m, Y_test_m)

print(f"Accuracy: {accuracy}")

Y_pred_raw = logreg_m.predict(X_test_m)

Y_pred = np.argmax(Y_pred_raw, axis=1)
Y_true = np.argmax(Y_test_m, axis=1)

print(f"F1 Score: {f1_score(Y_true, Y_pred, average='weighted')}")

In [ ]:
conf_mat = pd.DataFrame(confusion_matrix(Y_true, Y_pred),
                        index=["Fire", "Water", "Grass", "NA"],
                        columns=["Fire", "Water", "Grass", "NA"])
sn.heatmap(conf_mat, annot=True)


### Observations <a name="obs2">

We were able to obtain an accuracy of about 95% as well as an F1 Score of 97%, which is fantastic. Looking at the confusion matrix, we can see very good testing accuracy with select few misclassifications. Let's popoulate the observations table:
    
| Model | Observations | Accuracy | F1 |
|:----- |:------------| :--------| :--- |
|Logistic Regression (Binary) | Fairly high accuracy, many false positives in the confusion matrix | 90% |60% |
| Logistic Regression (Multi-class)| Very high accuracy, very few misclassifications |95%| 97% |
     
    
---

## $\mathcal{Fin}$

Congrats on making it to the end of this Notebook! Our next activity is to create a Logistic Regression Classifier in **Pytorch** to predict whether or not you should buy an SUV.
<img src="images/umaru.png" alt="cannot display image" style="width=700px">